In [1]:
import re
import string

import requests

import pandas as pd

In [9]:
import pymongo
client = pymongo.MongoClient('mongodb://donkey:27117/')
db = client.sphere_kaggle

db.unrecognized_locations.create_index('user_id', unique=True)

u'user_id_1'

In [10]:
train_users_data = pd.read_csv("./daemon/files/twitter_train.csv")
train_users_ids = list(train_users_data.twitter_id)

In [11]:
# Input your user name
GEO_USER_NAME = "kopbob"


def geoname_serializer(geoname):
    return {
        "lat": float(geoname.get("lat")),
        "lng": float(geoname.get("lng")),
        "name": geoname.get("name"),
        "countryCode": geoname.get("countryCode"),
    }

def fetch_geodata_by_location(location):
    payload = {'q': location, 'username': GEO_USER_NAME, 'maxRows': 1}
    search_results = requests.get("http://api.geonames.org/searchJSON",
                                  params=payload).json()

    geonames = search_results.get('geonames')
    if not geonames:
        return None
    
    top_related = geonames[0]
    
    return geoname_serializer(top_related)

In [42]:


train_users_cursor = db.users.find({"id":{"$in": train_users_ids}})

for user in train_users_cursor:
    if user.get('location') and not user.get("geo_location"):
        geodata = fetch_geodata_by_location(user.get('location'))
        if geodata:
            print user.get('id'), ": update", geodata
            db.users.update_one({'_id': user.get('_id')},
                                {"$set": {"geo_location": geodata}})
        else:
            try:
                db.unrecognized_locations.insert_one({
                        'user_id': user.get('id'),
                        'location': user.get('location')
                    })
                print user.get('id'), ": error", user.get('location')
            except pymongo.errors.DuplicateKeyError as e:
                pass
                
        
        

2495181 : update {'lat': 33.749, 'lng': -84.38798, 'name': u'Atlanta', 'countryCode': u'US'}
5520482 : update {'lat': 9.99299, 'lng': -84.12934, 'name': u'San Francisco', 'countryCode': u'CR'}
5530002 : update {'lat': 43.70229, 'lng': -72.28954, 'name': u'Hanover', 'countryCode': u'US'}
5715132 : update {'lat': 44.97997, 'lng': -93.26384, 'name': u'Minneapolis', 'countryCode': u'US'}
9699312 : update {'lat': 28.53834, 'lng': -81.37924, 'name': u'Orlando', 'countryCode': u'US'}
10143902 : update {'lat': 42.3751, 'lng': -71.10561, 'name': u'Cambridge', 'countryCode': u'US'}
10485842 : update {'lat': 19.42847, 'lng': -99.12766, 'name': u'Mexico City', 'countryCode': u'MX'}
10588142 : update {'lat': 44.8651, 'lng': -93.4209, 'name': u'Hampton Inn Mpls-Eden Prairie', 'countryCode': u'US'}
10745962 : update {'lat': 46.09454, 'lng': -64.7965, 'name': u'Moncton', 'countryCode': u'CA'}
10915772 : update {'lat': 39.00027, 'lng': -105.50083, 'name': u'Colorado', 'countryCode': u'US'}
11339662 : u

In [43]:
db.users.find({"geo_location": {"$exists": True}}).count()

1347

In [44]:
db.unrecognized_locations.find().count()

2152

In [45]:
1347 + 2152

3499

In [58]:
db.users.aggregate([
        {
            "$match": {
                "geo_location":{"$ne": None},
                "geo_location.countryCode": {"$in": ['CA', 'LA']},
            }
        },
        
    ])

In [59]:
list(_)

[{u'_id': ObjectId('566eb6bcc6de3b3dd8cab313'),
  u'contributors_enabled': False,
  u'created_at': u'Sun Dec 22 16:55:31 +0000 2013',
  u'default_profile': False,
  u'default_profile_image': False,
  u'description': u'Word Sorceress, editor, and author of Dragon Whisperer and After the Dragon Raid. Proficient in curse-speak. Tequila aficionado. Rage-monster. Mother. Geek.',
  u'entities': {u'description': {u'urls': []},
   u'url': {u'urls': [{u'display_url': u'thodestool.com',
      u'expanded_url': u'http://www.thodestool.com',
      u'indices': [0, 22],
      u'url': u'http://t.co/aZEug1fxl8'}]}},
  u'favourites_count': 9436,
  u'follow_request_sent': None,
  u'followers_count': 630,
  u'following': None,
  u'friends_count': 612,
  u'geo_enabled': False,
  u'geo_location': {u'countryCode': u'CA',
   u'lat': 43.4668,
   u'lng': -80.51639,
   u'name': u'Waterloo'},
  u'has_extended_profile': False,
  u'id': 2257928192L,
  u'id_str': u'2257928192',
  u'is_translation_enabled': False,
  